This is working okok

In [50]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
import joblib

# Step 1: Load the dataset
df = pd.read_excel("/content/drive/MyDrive/Hackron/NewDatasetForTraining.xlsx")

# Step 2: Handle missing values
categorical_features = ['Item names', 'Item identifier', 'Item type', 'Item fat content',
                        'Customer type', 'Seasonlity', 'Weather condition', 'Festive Season','Purchase frequency','Consumption time','Discounts and promotions','Time of day','Day of the week']
numerical_features = ['Item weight', 'Sales', 'Stock availablity',
                      'Competitor price','Demand surge', 'Supplier cost', 'Delivery cost', 'Warehouse stock',
                        'Ratings']

# Fill missing values
for col in categorical_features:
    df[col] = df[col].fillna("Unknown")
for col in numerical_features:
    df[col] = df[col].fillna(df[col].median())

# Encode categorical variables
label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Store encoders for inverse transform if needed

# Step 3: Feature Engineering
df['Days to Expiry'] = (pd.to_datetime(df['Expiry Date']) - pd.to_datetime(df['Current Date'])).dt.days
df.drop(columns=['Current Date', 'Expiry Date'], inplace=True)

# Step 4: Define Features and Target Variable
X = df.drop(columns=['Price'])
y = df['Price']

# Step 5: Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 6: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Step 7: Build the Neural Network Model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Step 8: Train the Model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Step 9: Evaluate the Model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R² Score: {r2}")

# Step 10: Save the Model
model.save("dynamic_pricing_nn_model.h5")
joblib.dump(scaler, "scaler.pkl")


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


213/213 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 46235.3242 - mae: 125.5217 - val_loss: 1879.0251 - val_mae: 34.9973
Epoch 2/100
213/213 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2096.0286 - mae: 34.9236 - val_loss: 952.2462 - val_mae: 25.3515
Epoch 3/100
213/213 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1359.0543 - mae: 27.3937 - val_loss: 593.3502 - val_mae: 19.4122
Epoch 4/100
213/213 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1082.8864 - mae: 22.7658 - val_loss: 341.8879 - val_mae: 13.5580
Epoch 5/100
213/213 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 884.2598 - mae: 19.4975 - val_loss: 264.5878 - val_mae: 11.4995
Epoch 6/100
213/213 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 824.0749 - mae: 18.5422 - val_loss: 221.0939 - val_mae: 10.2943
Epoch 7/100
213/213 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 818.3757 - mae: 17.7453 - val_loss: 282.9474 - val_mae: 11.0985
Epoch 8/100
213/213 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 783.5278 - mae: 17.0526 - val_loss: 197.3735 - val_mae: 9.1869
Epoch

MAE: 7.2338650843452
RMSE: 11.592683537052332
R² Score: 0.9965310801714515


['scaler.pkl']

In [51]:
def predict_price(sample_input):
    input_data = {}

    # Encode categorical features
    for col in categorical_features:
        if col in sample_input and sample_input[col] in label_encoders[col].classes_:
            input_data[col] = label_encoders[col].transform([sample_input[col]])[0]
        else:
            input_data[col] = 0  # Default value if unseen category

    # Convert numerical features
    for col in numerical_features + ['Days to Expiry']:
        if col in sample_input:
            value = sample_input[col]
            if isinstance(value, str) and '%' in value:
                value = float(value.strip('%')) / 100  # Convert percentage to decimal
            elif isinstance(value, str) and ':' in value:
                hours, minutes = map(int, value.split(':'))
                value = hours + minutes / 60  # Convert time to float
            elif isinstance(value, str):
                try:
                    value = float(value)
                except ValueError:
                    value = 0  # Default for unexpected string values
            input_data[col] = value

    # Convert to DataFrame
    input_df = pd.DataFrame([input_data])

    # Ensure column order matches training data
    input_df = input_df[X.columns]

    # Scale the data
    input_scaled = scaler.transform(input_df)

    # Predict price
    predicted_price = model.predict(input_scaled)[0][0]
    return predicted_price

# Sample Input
sample_input = {
    "Item names": "Apple",
    "Item identifier": "ITEM0057",
    "Item type": "Fruits and Vegetables",
    "Item fat content": "Low Fat",
    "Customer type": "Urban",
    "Seasonlity": "Medium",
    "Weather condition": "Cold",
    "Festive Season": "Diwali",
    "Item weight": 1.5,
    "Sales": 150,
    "Purchase frequency": 10,  # Converted from 'Daily'
    "Stock availablity": 500,
    "Consumption time": 8,  # Converted from 'Morning'
    "Competitor price": 180,
    "Discounts and promotions": "26%",  # Percentage conversion
    "Demand surge": 1.67,
    "Supplier cost": 133.9,
    "Delivery cost": 15,
    "Warehouse stock": 490,
    "Time of day": "10:00",  # Needs conversion
    "Day of the week": "Sunday",  # Needs encoding
    "Ratings": 4,
    "Days to Expiry": 31
}

# Predict
predicted_price = predict_price(sample_input)
print(f"Predicted Price: {predicted_price}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
Predicted Price: 191.40089416503906


This i am trying with more dense layers

In [49]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
import joblib

# Step 1: Load the dataset
df = pd.read_excel("/content/drive/MyDrive/Hackron/NewDatasetForTraining.xlsx")

# Step 2: Handle missing values
categorical_features = ['Item names', 'Item identifier', 'Item type', 'Item fat content',
                        'Customer type', 'Seasonlity', 'Weather condition', 'Festive Season','Purchase frequency','Consumption time','Discounts and promotions','Time of day','Day of the week']
numerical_features = ['Item weight', 'Sales', 'Stock availablity',
                      'Competitor price','Demand surge', 'Supplier cost', 'Delivery cost', 'Warehouse stock',
                        'Ratings']

# Fill missing values
for col in categorical_features:
    df[col] = df[col].fillna("Unknown")
for col in numerical_features:
    df[col] = df[col].fillna(df[col].median())

# Encode categorical variables
label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Store encoders for inverse transform if needed

# Step 3: Feature Engineering
df['Days to Expiry'] = (pd.to_datetime(df['Expiry Date']) - pd.to_datetime(df['Current Date'])).dt.days
df.drop(columns=['Current Date', 'Expiry Date'], inplace=True)

# Step 4: Define Features and Target Variable
X = df.drop(columns=['Price'])
y = df['Price']

# Step 5: Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 6: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Step 7: Build the Optimized Neural Network Model
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer
])

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Step 8: Train the Model with Early Stopping
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=1)

# Step 9: Evaluate the Model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R² Score: {r2}")

# Step 10: Save the Model
model.save("dynamic_pricing_nn_model1.h5")
joblib.dump(scaler, "scaler1.pkl")
joblib.dump(label_encoders, "label_encoders1.pkl")

# Step 11: Test with Sample Input
def predict_price(sample_input):
    # Convert categorical values using stored LabelEncoders
    input_data = {}
    for col in categorical_features:
        if col in sample_input:
            input_data[col] = label_encoders[col].transform([sample_input[col]])[0]

    # Add numerical features
    for col in numerical_features + ['Days to Expiry']:
        input_data[col] = sample_input[col]

    # Convert to DataFrame
    input_df = pd.DataFrame([input_data])

    # Ensure column order matches training data
    input_df = input_df[X.columns]

    # Scale numerical values
    input_scaled = scaler.transform(input_df)

    # Predict price
    predicted_price = model.predict(input_scaled)[0][0]
    return predicted_price

# Example input
sample_input = {
    "Item names": "Apple",
    "Item identifier": "ITEM0057",
    "Item type": "Fruits and Vegetables",
    "Item fat content": "Low Fat",
    "Customer type": "Urban",
    "Seasonlity": "Medium",
    "Weather condition": "Cold",
    "Festive Season": "Diwali",
    "Item weight": 1.5,
    "Sales": 150,
    "Purchase frequency": "Daily",  # Converted from 'Daily'
    "Stock availablity": 500,
    "Consumption time": "Morning",  # Converted from 'Morning'
    "Competitor price": 180,
    "Discounts and promotions": "26%",  # Percentage conversion
    "Demand surge": 1.67,
    "Supplier cost": 133.9,
    "Delivery cost": 15,
    "Warehouse stock": 490,
    "Time of day": "10:00",  # Needs conversion
    "Day of the week": "Sunday",  # Needs encoding
    "Ratings": 4,
    "Days to Expiry": 31
}

predicted_price = predict_price(sample_input)
print(f"Predicted Price: {predicted_price}")

Epoch 1/200


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


213/213 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - loss: 42255.1406 - mae: 121.3666 - val_loss: 4995.5020 - val_mae: 41.6075
Epoch 2/200
213/213 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 3121.3379 - mae: 38.0503 - val_loss: 821.2012 - val_mae: 20.4116
Epoch 3/200
213/213 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2334.7703 - mae: 31.2501 - val_loss: 1233.1832 - val_mae: 19.1384
Epoch 4/200
213/213 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2400.0315 - mae: 30.6981 - val_loss: 1647.8763 - val_mae: 19.3396
Epoch 5/200
213/213 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2218.1948 - mae: 29.3169 - val_loss: 1074.1049 - val_mae: 17.0355
Epoch 6/200
213/213 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1760.4718 - mae: 26.6005 - val_loss: 693.4694 - val_mae: 16.6974
Epoch 7/200
213/213 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1582.4132 - mae: 25.2400 - val_loss: 823.7384 - val_mae: 15.8073
Epoch 8/200
213/213 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1698.6548 - mae: 25.3160 - val_loss: 832.2961 - val_mae: 15.51

MAE: 12.028664179431692
RMSE: 17.16648875155216
R² Score: 0.992393423304892


ValueError: y contains previously unseen labels: '10:00'

In [55]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
import joblib

# Load the dataset
df = pd.read_excel("/content/drive/MyDrive/Hackron/NewDatasetForTraining.xlsx")

# Define categorical and numerical features
categorical_features = ['Item names', 'Item identifier', 'Item type', 'Item fat content',
                        'Customer type', 'Seasonlity', 'Weather condition', 'Festive Season',
                        'Purchase frequency', 'Consumption time', 'Discounts and promotions',
                        'Day of the week']

numerical_features = ['Item weight', 'Sales', 'Stock availablity', 'Competitor price',
                      'Demand surge', 'Supplier cost', 'Delivery cost', 'Warehouse stock',
                      'Ratings']

# Fill missing values
for col in categorical_features:
    df[col] = df[col].fillna("Unknown")
for col in numerical_features:
    df[col] = df[col].fillna(df[col].median())

# Encode categorical variables
label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Convert Dates to Days to Expiry
df['Days to Expiry'] = (pd.to_datetime(df['Expiry Date']) - pd.to_datetime(df['Current Date'])).dt.days
df.drop(columns=['Current Date', 'Expiry Date'], inplace=True)

# Convert 'Time of day' into sin & cos features
df['Hour'] = pd.to_datetime(df['Time of day'], format='%H:%M').dt.hour
df['Time_sin'] = np.sin(2 * np.pi * df['Hour'] / 24)
df['Time_cos'] = np.cos(2 * np.pi * df['Hour'] / 24)
df.drop(columns=['Time of day', 'Hour'], inplace=True)

# Define Features and Target Variable
X = df.drop(columns=['Price'])
y = df['Price']

# Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Build the Optimized Neural Network Model
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer
])

# Compile the model with optimized learning rate
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005), loss='mse', metrics=['mae'])

# Train the Model with Early Stopping
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=300, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=1)

# Evaluate the Model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R² Score: {r2}")

# Save the Model & Encoders
model.save("final_dynamic_pricing_nn_model.h5")
joblib.dump(scaler, "final_scaler.pkl")
joblib.dump(label_encoders, "final_label_encoders.pkl")

# Function to Predict Price for New Input
def predict_price(sample_input):
    input_data = {}

    # Encode categorical features
    for col in categorical_features:
        if col in sample_input:
            if sample_input[col] in label_encoders[col].classes_:
                input_data[col] = label_encoders[col].transform([sample_input[col]])[0]
            else:
                input_data[col] = 0  # Default if unseen category

    # Convert numerical features
    for col in numerical_features + ['Days to Expiry']:
        if col in sample_input:
            value = sample_input[col]
            if isinstance(value, str) and '%' in value:
                value = float(value.strip('%')) / 100  # Convert percentage to decimal
            elif isinstance(value, str):
                try:
                    value = float(value)
                except ValueError:
                    value = 0  # Default for unexpected string values
            input_data[col] = value

    # Convert 'Time of day' into sin & cos
    if 'Time of day' in sample_input:
        hours, minutes = map(int, sample_input['Time of day'].split(':'))
        input_data["Time_sin"] = np.sin(2 * np.pi * hours / 24)
        input_data["Time_cos"] = np.cos(2 * np.pi * hours / 24)

    # Convert to DataFrame and ensure column order
    input_df = pd.DataFrame([input_data])
    input_df = input_df[X.columns]

    # Scale the data
    input_scaled = scaler.transform(input_df)

    # Predict price
    predicted_price = model.predict(input_scaled)[0][0]
    return predicted_price

# Example Input
sample_input = {
    "Item names": "Tomato",
    "Item identifier": "ITEM0057",
    "Item type": "Fruits and Vegetables",
    "Item fat content": "Low Fat",
    "Customer type": "Urban",
    "Seasonlity": "Medium",
    "Weather condition": "Cold",
    "Festive Season": "Diwali",
    "Item weight": 1.5,
    "Sales": 800,
    "Purchase frequency": "Daily",
    "Stock availablity": 50,
    "Consumption time": "Morning",
    "Competitor price": 60,
    "Discounts and promotions": "20%",
    "Demand surge": 1.67,
    "Supplier cost": 38,
    "Delivery cost": 10,
    "Warehouse stock": 350,
    "Time of day": "10:00",
    "Day of the week": "Sunday",
    "Ratings": 4,
    "Days to Expiry": 23
}

predicted_price = predict_price(sample_input)
print(f"Predicted Price: {predicted_price}")


Epoch 1/300


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - loss: 52308.6992 - mae: 140.8433 - val_loss: 22841.6816 - val_mae: 95.4489
Epoch 2/300
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4397.1484 - mae: 43.0921 - val_loss: 12413.6494 - val_mae: 49.2171
Epoch 3/300
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 2586.6516 - mae: 31.2353 - val_loss: 6230.8833 - val_mae: 43.8650
Epoch 4/300
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2441.9006 - mae: 29.8316 - val_loss: 1499.3256 - val_mae: 24.8660
Epoch 5/300
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1743.7781 - mae: 26.8097 - val_loss: 405.4280 - val_mae: 14.4055
Epoch 6/300
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1900.8013 - mae: 27.4667 - val_loss: 543.4158 - val_mae: 14.7490
Epoch 7/300
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1477.9669 - mae: 24.4874 - val_loss: 510.4757 - val_mae: 14.0566
Epoch 8/300
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1718.6238 - mae: 25.8467 - val_loss: 577.0232 - val_mae: 14

MAE: 10.785310528923482
RMSE: 16.07202925741264
R² Score: 0.9933324277470581
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
Predicted Price: 52.778724670410156
